In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

import unidecode
import string
import random
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
 
import pdb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
 
all_characters = string.printable
n_characters = len(all_characters)

# A Christmas Carol @ https://www.gutenberg.org/files/24022/24022.zip


df = pd.read_csv('./drive/MyDrive/charles_dickens.csv')
df = df.loc[: , 'lines']
df.to_csv('./text_files/rnm.txt',index=False,header=False)

file = unidecode.unidecode(open('./text_files/rnm.txt').read())
file_len = len(file)
print('file_len =', file_len)

--2023-04-18 01:38:37--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 34.197.169.215, 54.204.89.5, 3.220.109.55, ...
Connecting to piazza.com (piazza.com)|34.197.169.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2023-04-18 01:38:38--  https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving cdn-uploads.piazza.com (cdn-uploads.piazza.com)... 108.138.246.58, 108.138.246.49, 108.138.246.41, ...
Connecting to cdn-uploads.piazza.com (cdn-uploads.piazza.com)|108.138.246.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[===================>]   1.46M  2.01M

In [ ]:
chunk_len = 200
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]
  
print(random_chunk())

 Trotwood, is that all the history?"
' said Agnes.
"'I hope it's enough, child,' said my aunt."
"'If there had been more money to lose, it wouldn't have been all, I dare say."
"Betsey would have contri


In [ ]:
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return tensor

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers

    self.embedding = nn.Embedding(output_size, hidden_size)
    self.gru = nn.GRU(input_size, hidden_size, n_layers)
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input_char, hidden):
    # by reviewing the documentation, construct a forward function that properly uses the output
    # of the GRU

    output = self.embedding(input_char).view(1, 1, -1)
    output = F.relu(output)
    output, hidden = self.gru(output, hidden)
    probabilities = self.out(output[0])
    
    return probabilities, hidden

  def init_hidden(self):
    return torch.zeros(self.n_layers, 1, self.hidden_size)

In [ ]:
def random_training_set():
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

In [ ]:
# NOTE: decoder_optimizer, decoder, and criterion are defined below as global variables
def train(input_tensor, target_tensor):
  # initialize hidden layers, set up gradient and loss

  decoder_optimizer.zero_grad()
  hidden = decoder.init_hidden()
  loss = 0
  
  input_length = input_tensor.size(0)
  target_length = target_tensor.size(0)

  decoder_input = input_tensor
  decoder_hidden = decoder.init_hidden()

  # Teacher forcing: Feed the target as the next input
  for di in range(target_length):
    decoder_output, decoder_hidden = decoder(decoder_input[di], decoder_hidden)
    loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))

  loss.backward()

  decoder_optimizer.step()

  return loss.item() / target_length

In [ ]:
def sample_outputs(output, temperature):
    """Takes in a vector of unnormalized probability weights and samples a character from the distribution"""
    return torch.multinomial(torch.exp(output / temperature), 1)

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  # initialize hidden state, others

  with torch.no_grad():
    decoder_input = torch.tensor(char_tensor(prime_str), device=device)
    decoder_hidden = decoder.init_hidden()

    decoded_chars = []

    for i in range(predict_len):
      decoder_output, decoder_hidden = decoder(decoder_input[i], decoder_hidden)
      predicted_char_t = sample_outputs(decoder_output, temperature).clone().detach()
      decoded_chars.append(all_characters[predicted_char_t.item()])
      if i >= len(prime_str)-1: decoder_input = torch.cat((decoder_input, predicted_char_t.squeeze(0)))

    return decoded_chars


---

## Main Loop

---

In [ ]:
import time
n_epochs = 20000
print_every = 50
plot_every = 10
predict_length = 100
hidden_size = 100
n_layers = 7
lr = 0.001
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[Time: %s (%d %d%%) Loss: %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      str1 = ""
      print('Wh',str1.join(evaluate('Wh',predict_length)),'\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[Time: 47.13232898712158 (50 0%) Loss: 3.0353]
Wh hdhtd hhnutn nen  naeekraenae y Is.td"tgt ohi anh cctiee" y heoeaf sei    t hr d'ethinoonn oheui   w 



<ipython-input-8-eb901ae2cafd>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = torch.tensor(char_tensor(prime_str), device=device)


[Time: 84.891042470932 (100 0%) Loss: 3.0950]
Wh racagnmhp emst  hner herm o eiins   ah   w ntr
aeai ee'rakiig p.tuaTab  vo"c ti 'as sdeMteetenndenir 

[Time: 124.6094183921814 (150 0%) Loss: 3.0211]
Wh  gank a   enlt me  wh erd  endhs
" t  s  hw"a Iim- ohte Netm o rt  h c ni'tdsu rbi d"escl"a g!,o ele 

[Time: 162.99321699142456 (200 1%) Loss: 3.1683]
Wh rero 'osatoeilsfuv,  u" -gdchmfnu 
banh t wle bnmaedfie  )nieywe dtplihaendi  aptioeyrt nt neae feth 

[Time: 203.03389883041382 (250 1%) Loss: 3.3481]
Wh  n tetn'  tc a sai eermr d esimyarnesor  od' E badr sn e inh iaeaey.shoeat h eh  doht e n a hoc.iaos 

[Time: 241.98037695884705 (300 1%) Loss: 3.1496]
Wh cxl eg tyet wn  e  et eeeehoi dgsnn tf  d sdthmsot,hdy ruapan  "c a'ctop b h  elreco pmweo  a,dte ob 

[Time: 279.92183923721313 (350 1%) Loss: 3.3174]
Wh meenrerwa h?y, h  ity a   sam na i.oihi t niot tt.tea szdsnrsrnea   ih"n"o ehee me etscte  u  .h" u  

[Time: 318.21096634864807 (400 2%) Loss: 3.1074]
Wh wM  es e nit f e "eete

In [ ]:
start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra", "Even", "How", "What if"]
for i in range(len(start_strings)):
  start = random.randint(0,len(start_strings)-1)
  print("-----",start_strings[start], " -----")
  str1 = ""
  print(str1.join(evaluate(start_strings[start], 200)),'\n')

-----  he  -----
peoas ai ss  c t"
"Hl u evi  h  malranhwmn h aoa   hori uien ld aneahihitysnfgi d y c a e  t he  
 ohrlhet dhy   ewnne aifeimrotto"
"Folw   edws  an e cpssa netuuehethia"
"T"""I"' 
Sr tai es  ou  naci 

----- What if  -----


<ipython-input-8-eb901ae2cafd>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = torch.tensor(char_tensor(prime_str), device=device)


t ennn,. a amirgteolsncs o aoioihohentrg"
"He ges renermnedfh isigerracr ssi ssh in c  hfa htvh"
""?"
hplyInouo  euu" "Wt o c"
 I  nenlu  iscae a em eesn aya shh wf ylwee hrohginsthaeeuyh n tnelicsnoo 

----- Even  -----
igs?hi a.id saiat so wshfied.oee t tgutlwv  ks"
 e,osih nitrr  d-rinqfu  e eot"

,,n eare-ra aei , Ibni alu  spya  movld nr  to. ttw  g airyi lu ddpedr oet reieaschtgyrl tl gao  ihntea?eeahl rtwteeaf  

-----  G  -----
e.snnd ree o  rtsteresogo l ya bgtnae  erhf sa   hr   aeceahoeearr, a c?rct   wht imw.eoa g nwl  ersr ewdeI 
tdepo yod tmahwa rnnoagshddghe ailhunl   eeCfop  ne ilihos"T"'yyseesyd  pixgas . idl'e e ss 

-----  lo  -----
lnenogtn   eu saooeath  rnurs shraylne ieoeiaeaontsaw,arr esi eit.ddmueeare,m n o oiieentnd te  ow henh nin oitnua ya  tdo-   eohdre ;ehasseeooBgaf s u ih,sa heddh"
"""
do aoay dass  ntmatlg te ceuee  

-----  Th  -----
io oi  o easrrureef earohoeetbonnu Ntrh  e gwlri n eohtes icetoy,wtdhts  hodiio eeret oAr  rumaIid e ,g Buhnnso,foinea h